# Reddit Data Gathering
<span style="color:red">Reminder : This is a new database that is made upon the old database</span>
## Data Scraping

In [97]:
import praw
import pandas
import re
import numpy as np

In [3]:
reddit = praw.Reddit(client_id='aAcZ6aykDcIyaw',
                     client_secret='Y-zrtAoc0akVR-qf6SwEGcA0f2A',
                     user_agent='subred-data')

In [4]:
hot_posts = reddit.subreddit('india').hot(limit=1500)
new_posts = reddit.subreddit('india').new(limit=1500)
top_posts = reddit.subreddit('india').top(limit=1500)

In [10]:
posts =[]
posts.extend(hot_posts)
posts.extend(new_posts)
posts.extend(top_posts)

In [76]:
def get_comments(submission):
    submission.comment_sort = 'top'
    submission.comments.replace_more(limit=0)
    comments = submission.comments.list()[:10]
    comment_list = [re.sub("[^a-zA-Z ]+", "", comment.body) for comment in comments]
    return comment_list

In [135]:
details = []
count=0
for post in posts:
    count+=1
    details.append([post.id,post.title,post.author,post.link_flair_text,post.over_18,post.score,post.selftext,len(post.all_awardings),post.created_utc,post.category,post.num_comments,post.ups,get_comments(post)])


In [81]:
count

2539

#### Dataset creation from collected data

In [ ]:


import pandas as pd
df = pd.DataFrame(details,columns=['id','title','author','flare','over_18','score','selftext','awards','creation_time','category','num_comments','upvotes','comments'])
df.to_csv('raw.csv', index=False)
df.head()


In [99]:
list(set(df['category']))

[None]

In [101]:
del df['category']

In [102]:
del df['upvotes']


In [103]:
df.head()

,id,title,author,flare,over_18,score,selftext,awards,creation_time,num_comments,comments
0,g1zi21,Coronavirus (COVID-19) Megathread - News and U...,IAmMohit,Coronavirus,False,190,###[Covid-19 Fundraisers & Donation Links](htt...,0,1.586981e+09,3543,[Covid Fundraisers Donation Linkshttpsamnesty...
1,fzub9j,"Announcing r/IndiaMeme, our own sub for memes ...",ppatra,Announcement,False,300,HELLO YOU NICE PEOPLE. WE GOT REALLY TIRED OF ...,1,1.586687e+09,86,[This is the right way of introducing a sub ra...
2,g7vfmg,PM Cares Fund,sudden_dust,Politics,False,1208,,0,1.587829e+09,109,[The greatest crisis in over a century Need of...
3,g7qeec,"""Doland, Don't do Press Conference...""",pitchwin,Politics,False,2496,,0,1.587805e+09,69,"[Dolund, Also blame the Mexicans from time to ..."
4,g83rwt,Jharkhand Police swiftly act to book communal ...,hauntin,Coronavirus,False,105,,0,1.587858e+09,9,"[This is unbelievable, BJP out of power right,..."


#### Listing unique flare values

In [105]:
flares = list(set(df['flare']))
flares

['OC',
 'Politics -- Source in comments',
 'Original Comics',
 '[R]eddiquette',
 'Sports',
 'CAA-NRC-NPR',
 'Announcement',
 'Entertainment',
 'Misleading Headline ',
 'Politics [Megathread]',
 'Totally real',
 'Unverified',
 'Non-Political',
 'Scheduled',
 'Zoke Tyme',
 'Food',
 'Politics',
 'Lifehacks',
 None,
 'AskIndia',
 'Business/Finance',
 'Policy & Economy',
 '/r/all',
 'CAA-NRC',
 'On Internet Shutdowns',
 'Science/Technology',
 'TIL',
 'Policy/Economy',
 'Photography',
 'Policy/Economy -2017 Article ',
 'r/all',
 'Coronavirus',
 'Demonetization',
 'AMA',
 'Official Sadness Thread',
 'Misleading']

#### Desired flare defining 

In [ ]:
flare_classes=['Politics',
'Non-Political',
'AskIndia',
'Policy/Economy',
'Business/Finance',
'Science/Technology',
'Scheduled',
'Sports',
'Food',
'Photography',
'CAA-NRC-NPR',
'Coronavirus']

In [107]:
df.describe()

,score,awards,creation_time,num_comments
count,2538.000000,2538.000000,2.538000e+03,2538.000000
mean,1035.367218,0.218676,1.576455e+09,87.595745
std,1659.932463,0.715732,2.251530e+07,232.066137
min,0.000000,0.000000,1.429104e+09,0.000000
25%,12.250000,0.000000,1.576779e+09,3.000000
50%,78.500000,0.000000,1.587561e+09,17.000000
75%,1863.750000,0.000000,1.587719e+09,115.000000
max,19711.000000,15.000000,1.587877e+09,6911.000000


#### Removal of duplication in database based on id

In [108]:
df_unique=df.drop_duplicates('id',keep='last')

In [109]:
df_unique

,id,title,author,flare,over_18,score,selftext,awards,creation_time,num_comments,comments
0,g1zi21,Coronavirus (COVID-19) Megathread - News and U...,IAmMohit,Coronavirus,False,190,###[Covid-19 Fundraisers & Donation Links](htt...,0,1.586981e+09,3543,[Covid Fundraisers Donation Linkshttpsamnesty...
1,fzub9j,"Announcing r/IndiaMeme, our own sub for memes ...",ppatra,Announcement,False,300,HELLO YOU NICE PEOPLE. WE GOT REALLY TIRED OF ...,1,1.586687e+09,86,[This is the right way of introducing a sub ra...
714,g59p38,Lockdown forced us to cut our own hair 😧,bluesbaba,Coronavirus,False,1206,,3,1.587449e+09,163,[Bhaari video ekdum Baba killing it with the s...
748,g57gd7,Muslims are feeding you and carrying your dead...,hauntin,Politics,False,1074,,0,1.587439e+09,158,[Can someone tell me what world are we living ...
750,g5a979,Palghar lynching: When police watches helpless...,aviakki1,Non-Political,False,517,,0,1.587451e+09,58,[The Wire had already reported two years ago a...
...,...,...,...,...,...,...,...,...,...,...,...
2533,aj176g,What my class (Austria 11th school level) is t...,MohaTi,Non-Political,False,1512,,1,1.548260e+09,349,[Sweet Someone mentioned Siddhartha Gautama In...
2534,8manku,This politician brought Virat Kohli as his chi...,Cricketloverbybirth,Politics,False,1506,,0,1.527347e+09,126,[WTF Who has been captaining the Indian side...
2535,b54uf5,Congress Ads are in uTorrent now,thescientificindian,Politics,False,1509,,0,1.553480e+09,206,[Use qbittorrent manIts open source and has no...
2536,6t0x9r,View from my plane while i leave my house to p...,None,Non-Political,False,1503,,0,1.502456e+09,117,"[Rich af, Another spoilt brat who just likes t..."


#### Flare distribution 

In [111]:
df_unique['flare'].value_counts()

Non-Political                     507
Politics                          414
Coronavirus                       348
AskIndia                          142
Policy/Economy                     69
Business/Finance                   58
Photography                        48
Sports                             33
[R]eddiquette                      29
Science/Technology                 27
Food                               23
Unverified                         12
Scheduled                          12
CAA-NRC                             8
CAA-NRC-NPR                         7
r/all                               3
Policy & Economy                    2
AMA                                 2
/r/all                              2
Zoke Tyme                           1
Politics [Megathread]               1
TIL                                 1
Politics -- Source in comments      1
OC                                  1
Official Sadness Thread             1
Misleading                          1
Original Com

delete all entries which don't fall under 12 flares

#### Removing undesired flare rows 

In [113]:
db_filter = df_unique.loc[df['flare'].isin(flare_classes)]

#### Handling NAN values

In [114]:
db_filter.fillna('', inplace=True)

/home/sjsingh/.local/lib/python3.6/site-packages/pandas/core/frame.py:4259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


In [122]:
db_filter.head()

,id,title,author,flare,over_18,score,selftext,awards,creation_time,num_comments,comments
0,g1zi21,Coronavirus (COVID-19) Megathread - News and U...,IAmMohit,Coronavirus,False,190,###[Covid-19 Fundraisers & Donation Links](htt...,0,1.586981e+09,3543,[Covid Fundraisers Donation Linkshttpsamnesty...
714,g59p38,Lockdown forced us to cut our own hair 😧,bluesbaba,Coronavirus,False,1206,,3,1.587449e+09,163,[Bhaari video ekdum Baba killing it with the s...
748,g57gd7,Muslims are feeding you and carrying your dead...,hauntin,Politics,False,1074,,0,1.587439e+09,158,[Can someone tell me what world are we living ...
750,g5a979,Palghar lynching: When police watches helpless...,aviakki1,Non-Political,False,517,,0,1.587451e+09,58,[The Wire had already reported two years ago a...
757,g5bddv,Mobile Internet speed is slow in India than Pa...,RemarkableWork,Science/Technology,False,299,,0,1.587457e+09,48,[Jio has already been giving g speeds with a g...


In [118]:
db_filter.dtypes

id                object
title             object
author            object
flare             object
over_18             bool
score              int64
selftext          object
awards             int64
creation_time    float64
num_comments       int64
comments          object
dtype: object

#### Joining title and selftext to make a combined corpous of data

In [126]:
db_filter['corpous'] = [title+''+text for title,text in np.c_[db_filter['title'],db_filter['selftext']]]

In [127]:
db_filter.head()

,id,title,author,flare,over_18,score,selftext,awards,creation_time,num_comments,comments,corpous
0,g1zi21,Coronavirus (COVID-19) Megathread - News and U...,IAmMohit,Coronavirus,False,190,###[Covid-19 Fundraisers & Donation Links](htt...,0,1.586981e+09,3543,[Covid Fundraisers Donation Linkshttpsamnesty...,Coronavirus (COVID-19) Megathread - News and U...
714,g59p38,Lockdown forced us to cut our own hair 😧,bluesbaba,Coronavirus,False,1206,,3,1.587449e+09,163,[Bhaari video ekdum Baba killing it with the s...,Lockdown forced us to cut our own hair 😧
748,g57gd7,Muslims are feeding you and carrying your dead...,hauntin,Politics,False,1074,,0,1.587439e+09,158,[Can someone tell me what world are we living ...,Muslims are feeding you and carrying your dead...
750,g5a979,Palghar lynching: When police watches helpless...,aviakki1,Non-Political,False,517,,0,1.587451e+09,58,[The Wire had already reported two years ago a...,Palghar lynching: When police watches helpless...
757,g5bddv,Mobile Internet speed is slow in India than Pa...,RemarkableWork,Science/Technology,False,299,,0,1.587457e+09,48,[Jio has already been giving g speeds with a g...,Mobile Internet speed is slow in India than Pa...


#### Converting boolean to integer values in Over_18 column

In [130]:
db_filter.over_18=db_filter.over_18.astype(int)

In [131]:
db_filter.over_18.value_counts()

0    1680
1       8
Name: over_18, dtype: int64

In [133]:
del db_filter['selftext']

#### Handling newly added comment columns 

In [138]:
comment_col=[]
for post in db_filter['comments'].values:
    for comment in post:
        comment_col.append(' '.join([w for w in comment.split() if len(w)<16]))


In [155]:
db_filter['comments'].value = np.array(comment_col)
comment_col

['Covid Fundraisers Donation via Amnesty International This link Migrant Workers DayLabourers Other Vulnerable Groups Urban Poor Transgender Community Wastepickers and Sanitation Workers Healthcare Workers and Doctors Older Persons Children and Animal Care I am looking for volunteers who are willing to create and update their state level threads for the time of lockdown Updates will mostly consist of latest news in those states with respect to Coronavirus and ongoing lockdownRequire Volunteers for following States Uttar Pradesh Jammu Kashmir and Ladakh Sikkim Manipur Mizoram Assam Meghalaya Tripura Arunachal Pradesh Jharkhand Goa Chattisgarh Haryana Himachal Pradesh Nagaland Andhra PradeshPlease reply under this comment with the states you can update daily Daily update will be a responsibility so please only volunteer if you think you will have time to maintain it in coming weeks ThanksOlder Threads Part Part Part',
 'So Goa tested just people and declared itself a Green Zone',
 'I dro

In [156]:
db_filter.head()

,id,title,author,flare,over_18,score,awards,creation_time,num_comments,comments,corpous
0,g1zi21,Coronavirus (COVID-19) Megathread - News and U...,IAmMohit,Coronavirus,0,190,0,1.586981e+09,3543,[Covid Fundraisers Donation Linkshttpsamnesty...,Coronavirus (COVID-19) Megathread - News and U...
714,g59p38,Lockdown forced us to cut our own hair 😧,bluesbaba,Coronavirus,0,1206,3,1.587449e+09,163,[Bhaari video ekdum Baba killing it with the s...,Lockdown forced us to cut our own hair 😧
748,g57gd7,Muslims are feeding you and carrying your dead...,hauntin,Politics,0,1074,0,1.587439e+09,158,[Can someone tell me what world are we living ...,Muslims are feeding you and carrying your dead...
750,g5a979,Palghar lynching: When police watches helpless...,aviakki1,Non-Political,0,517,0,1.587451e+09,58,[The Wire had already reported two years ago a...,Palghar lynching: When police watches helpless...
757,g5bddv,Mobile Internet speed is slow in India than Pa...,RemarkableWork,Science/Technology,0,299,0,1.587457e+09,48,[Jio has already been giving g speeds with a g...,Mobile Internet speed is slow in India than Pa...


## Feature Engineering

#### Mean of columns

In [161]:
mean_score = db_filter.score.mean()
mean_awards = db_filter.awards.mean()
mean_num_comments = db_filter.num_comments.mean()
print("Mean of score is : {}\nMean of awards is : {}\nMean of number of comments is : {}\n".format(mean_score,mean_awards,mean_num_comments))

Mean of score is : 1337.4662322274883
Mean of awards is : 0.29206161137440756
Mean of number of comments is : 103.55331753554502



#### Feature Scaling

In [162]:
db_filter.score = db_filter.score/db_filter.score.mean()

In [163]:
db_filter.num_comments = db_filter.num_comments/db_filter.num_comments.mean()

In [164]:
db_filter.awards = db_filter.awards/db_filter.awards.mean()

#### Saving dataset 

In [167]:
db_filter.to_csv('data/filtered.csv', index=False)